## Similarity function comparison

In [1]:
import os
#virtually move to parent directory
os.chdir("..")

import torch
from sentence_transformers import SentenceTransformer
from sklearn import metrics

import clip
import utils
import similarity

## Settings

In [2]:
similarity_fns = ["cos_similarity", "rank_reorder", "wpmi", "soft_wpmi"]
d_probes = ['cifar100_train', 'broden', 'imagenet_val', 'imagenet_broden']

clip_name = 'ViT-B/16'
target_name = 'resnet50'
target_layer = 'fc'
batch_size = 200
device = 'cuda'
pool_mode = 'avg'
save_dir = 'saved_activations'

In [3]:
model = SentenceTransformer('all-mpnet-base-v2')
clip_model, _ = clip.load(clip_name, device=device)

with open("data/imagenet_labels.txt", "r") as f:
    cls_id_to_name = f.read().split("\n")

# Cos similarities

In [4]:
concept_set = 'data/20k.txt'

with open(concept_set, 'r') as f:
    words = f.read().split('\n')

for similarity_fn in similarity_fns:
    for d_probe in d_probes:
        utils.save_activations(clip_name = clip_name, target_name = target_name, target_layers = [target_layer], 
                               d_probe = d_probe, concept_set = concept_set, batch_size = batch_size, 
                               device = device, pool_mode=pool_mode, save_dir = save_dir)

        save_names = utils.get_save_names(clip_name = clip_name, target_name = target_name,
                                          target_layer = target_layer, d_probe = d_probe,
                                          concept_set = concept_set, pool_mode=pool_mode,
                                          save_dir = save_dir)

        target_save_name, clip_save_name, text_save_name = save_names

        similarities, target_feats = utils.get_similarity_from_activations(target_save_name, clip_save_name, 
                                                                           text_save_name, 
                                                                           eval("similarity.{}".format(similarity_fn)),
                                                                           device=device)

        clip_preds = torch.argmax(similarities, dim=1)
        clip_preds = [words[int(pred)] for pred in clip_preds]

        clip_cos, mpnet_cos = utils.get_cos_similarity(clip_preds, cls_id_to_name, clip_model, model, device, batch_size)
        print("Similarity fn: {}, D_probe: {}".format(similarity_fn, d_probe))
        print("Clip similarity: {:.4f}, mpnet similarity: {:.4f}".format(clip_cos, mpnet_cos))

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 1/1 [00:01<00:00,  1.65s/it]


Similarity fn: cos_similarity, D_probe: cifar100_train
Clip similarity: 0.6484, mpnet similarity: 0.2756


100%|██████████| 1/1 [00:01<00:00,  1.81s/it]


Similarity fn: cos_similarity, D_probe: broden
Clip similarity: 0.6235, mpnet similarity: 0.2153


...

torch.Size([1000, 20000])
Similarity fn: soft_wpmi, D_probe: broden
Clip similarity: 0.7412, mpnet similarity: 0.3946


100%|██████████| 1000/1000 [00:04<00:00, 209.43it/s]


torch.Size([1000, 20000])
Similarity fn: soft_wpmi, D_probe: imagenet_val
Clip similarity: 0.7900, mpnet similarity: 0.5262


100%|██████████| 1000/1000 [00:04<00:00, 209.87it/s]


torch.Size([1000, 20000])
Similarity fn: soft_wpmi, D_probe: imagenet_broden
Clip similarity: 0.7900, mpnet similarity: 0.5239


# Accuracies

In [5]:
def get_topk_acc(sim, k=5):
    correct = 0
    for orig_id in range(1000):
        vals, ids = torch.topk(sim[orig_id], k=k)
        for idx in ids[:k]:
            correct += (int(idx)==orig_id)
    return (correct/1000)*100

def get_correct_rank_mean_median(sim):
    ranks = []
    for orig_id in range(1000):
        vals, ids = torch.sort(sim[orig_id], descending=True)
        
        ranks.append(list(ids).index(orig_id)+1)
        
    mean = sum(ranks)/len(ranks)
    median = sorted(ranks)[500]
    return mean, median

def get_auc(sim):
    max_sim, preds = torch.max(sim.cpu(), dim=1)
    gtruth = torch.arange(0, 1000)
    correct = (preds==gtruth)
    fpr, tpr, thresholds = metrics.roc_curve(correct, max_sim)
    auc = metrics.roc_auc_score(correct, max_sim)
    return auc

In [9]:
concept_set = 'data/imagenet_labels.txt'
with open(concept_set, 'r') as f: 
    words = (f.read()).split('\n')
    

for similarity_fn in similarity_fns:
    for d_probe in d_probes:
        utils.save_activations(clip_name = clip_name, target_name = target_name, target_layers = [target_layer], 
                               d_probe = d_probe, concept_set = concept_set, batch_size = batch_size, 
                               device = device, pool_mode=pool_mode, save_dir = save_dir)

        save_names = utils.get_save_names(clip_name = clip_name, target_name = target_name,
                                          target_layer = target_layer, d_probe = d_probe,
                                          concept_set = concept_set, pool_mode=pool_mode,
                  
                                          save_dir = save_dir)

        target_save_name, clip_save_name, text_save_name = save_names

        similarities, target_feats = utils.get_similarity_from_activations(target_save_name, clip_save_name, 
                                                                           text_save_name, 
                                                                           eval("similarity.{}".format(similarity_fn)),
                                                                           device=device)
        
        print("Similarity fn: {}, D_probe: {}".format(similarity_fn, d_probe))
        print("Top 1 acc: {:.2f}%, Top 5 acc: {:.2f}%".format(get_topk_acc(similarities, k=1),
                                                         get_topk_acc(similarities, k=5)))
        
        mean, median = get_correct_rank_mean_median(similarities)
        print("Mean rank of correct class: {:.2f}, Median rank of correct class: {}".format(mean, median))
        print("AUC: {:.4f}".format(get_auc(similarities)))



Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 1/1 [00:00<00:00,  9.83it/s]


Similarity fn: cos_similarity, D_probe: cifar100_train
Top 1 acc: 8.60%, Top 5 acc: 25.10%
Mean rank of correct class: 53.94, Median rank of correct class: 21
AUC: 0.5926


100%|██████████| 1/1 [00:00<00:00,  8.83it/s]


Similarity fn: cos_similarity, D_probe: broden
Top 1 acc: 5.70%, Top 5 acc: 21.30%
Mean rank of correct class: 63.92, Median rank of correct class: 24
AUC: 0.5710


...

torch.Size([1000, 1000])
Similarity fn: soft_wpmi, D_probe: broden
Top 1 acc: 70.70%, Top 5 acc: 90.00%
Mean rank of correct class: 4.80, Median rank of correct class: 1
AUC: 0.7856


100%|██████████| 1000/1000 [00:00<00:00, 1344.09it/s]


torch.Size([1000, 1000])
Similarity fn: soft_wpmi, D_probe: imagenet_val
Top 1 acc: 95.50%, Top 5 acc: 98.90%
Mean rank of correct class: 1.18, Median rank of correct class: 1
AUC: 0.9208


100%|██████████| 1000/1000 [00:00<00:00, 1253.33it/s]


torch.Size([1000, 1000])
Similarity fn: soft_wpmi, D_probe: imagenet_broden
Top 1 acc: 95.40%, Top 5 acc: 99.00%
Mean rank of correct class: 1.19, Median rank of correct class: 1
AUC: 0.9166
